In [3]:
%load_ext autoreload
%autoreload 2

import requests
import random
import numpy as np
import tqdm
from IPython.display import Image, display
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Fetch dataset

In [38]:
link = "http://lib.stat.cmu.edu/datasets/boston"
raw_text = requests.get(link).text # get the raw text
raw_text = raw_text.split('\n')[22:-1] # split into lines, discard description

raw_data = []
for i in range(0, len(raw_text), 2):
    # each row is split into 2 lines, so join them first
    row = raw_text[i] + raw_text[i+1]
    raw_data.append(row)

# split each row into columns and convert each cell -> string to float
raw_data = [[float(column) for column in row.split()] for row in raw_data]
raw_data = np.array(raw_data) # no error, implies equal length rows, error-free import

In [39]:
print(raw_data)

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 3.9690e+02 4.9800e+00 2.4000e+01]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 3.9690e+02 9.1400e+00 2.1600e+01]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 3.9283e+02 4.0300e+00 3.4700e+01]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 3.9690e+02 5.6400e+00 2.3900e+01]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 3.9345e+02 6.4800e+00 2.2000e+01]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 3.9690e+02 7.8800e+00 1.1900e+01]]
